**1. Implement a simple feedforward neural network from scratch using Python (without a deep learning library like TensorFlow or PyTorch).**

**Details:** Train your network on a small dataset (e.g., XOR dataset or Iris dataset) and include backpropagation for weight updates. Compare the results with an equivalent implementation using a library like TensorFlow/Keras.

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [49]:
DEFAULT_RANDOM_SEED = 42

def set_all_seeds(seed=DEFAULT_RANDOM_SEED):

    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    import random
    random.seed(seed)

    import os
    os.environ['PYTHONHASHSEED'] = str(seed)

set_all_seeds(seed=DEFAULT_RANDOM_SEED)

SimpleNN on iris dataset

In [50]:
class SimpleNN:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        self.W1 = np.random.rand(input_size, hidden_size)
        self.W2 = np.random.rand(hidden_size, output_size)
        self.learning_rate = learning_rate

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.hidden = self.sigmoid(np.dot(X, self.W1))
        self.output = self.sigmoid(np.dot(self.hidden, self.W2))
        return self.output

    def backward(self, X, y):
        output_error = y - self.output
        output_delta = output_error * self.sigmoid_derivative(self.output)

        hidden_error = output_delta.dot(self.W2.T)
        hidden_delta = hidden_error * self.sigmoid_derivative(self.hidden)

        self.W2 += self.hidden.T.dot(output_delta) * self.learning_rate
        self.W1 += X.T.dot(hidden_delta) * self.learning_rate

    def train(self, X, y, epochs):
        for _ in range(epochs):
            self.forward(X)
            self.backward(X, y)

iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

net = SimpleNN(input_size=4, hidden_size=5, output_size=3)
net.train(X_train, y_train, epochs=10000)

predictions = net.forward(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

accuracy = np.mean(predicted_classes == true_classes)
print(f"Accuracy of the custom NN on the Iris dataset: {accuracy:.2f}")

Accuracy of the custom NN on the Iris dataset: 0.63


iris dataset with an equivalent implementation using a library like TensorFlow/Keras.

In [4]:
iris = load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = (predicted == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy.item() * 100:.2f}%')

Accuracy: 96.67%


It can be seen that the result obtained using simpleNN, to put it mildly, is not particularly good, unlike the library implementation.

**3. Implement a network pruning technique to optimise a pre-trained neural network.**

**Details:** Train a simple dense neural network on a dataset (e.g., Fashion-MNIST) and prune weights below a certain threshold. Compare the performance and efficiency of the pruned network with the original.

In [5]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [51]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class ImprovedNN(nn.Module):
    def __init__(self):
        super(ImprovedNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ImprovedNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return correct / total

train(model, train_loader, criterion, optimizer)

original_accuracy = evaluate(model, test_loader)
print(f"Original Model - accuracy: {original_accuracy:.4f}")

def prune_weights(model, threshold):
    with torch.no_grad():
        for param in model.parameters():
            param.data[torch.abs(param.data) < threshold] = 0

prune_threshold = 0.05
prune_weights(model, prune_threshold)

pruned_accuracy = evaluate(model, test_loader)
print(f"pruned Model - accuracy: {pruned_accuracy:.4f}")

print(f"accuracy difference: {pruned_accuracy - original_accuracy:.4f}")

Original Model - accuracy: 0.8786
pruned Model - accuracy: 0.8757
accuracy difference: -0.0029


The results are normal, but unsatisfactory. Let's try to tune

In [52]:
def fine_tune(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

fine_tune(model, train_loader, criterion, optimizer)

pruned_fine_tuned_accuracy = evaluate(model, test_loader)
print(f"accuracy differnce after fine-tuning: {pruned_fine_tuned_accuracy - original_accuracy:.4f}")

accuracy differnce after fine-tuning: 0.0089


After tuning, we got a small increase in accuracy.